In [35]:
import scipy as sc
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]
import warnings
from statistics import mean, median
warnings.filterwarnings('ignore')
import math

In [36]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [126]:
h = 0.02
sample_size = 10
dt = 1  # period of descritization
p = 3  # Prony param: depth of the model

my_d = \
    [sum([k*math.exp(-h*i / k) * math.cos(4*math.pi*k*h*i + math.pi/k) for k in range(1, 4)]) for i in range(sample_size)]

my_d = [1.1387, 1.09506, 1.04571, 0.991055]
p = 2
sample_size = 4

Prony Method

In [137]:
x_left = np.array([[my_d[p-i+j-1] for i in range(p)] for j in range(sample_size - p)])
x_right = np.array([-my_d[p+j] for j in range(sample_size - p)])

model = LinearRegression()
model.fit(x_left, x_right)
print(my_d)
print()
print(x_left)
print()
print(x_right)

a_vector = np.linalg.solve(x_left.T.dot(x_left), x_left.T.dot(x_right))
z_vector = np.roots(np.append(1, a_vector))
f_vector = [1 / (2 * math.pi * dt) * math.atan(z_vector[i].imag / z_vector[i].real) for i in range(p)]
alfa_vector = [1 / dt * math.log(abs(z_vector[i])) for i in range(p)]
z_left = np.array([[z_vector[i] ** j for i in range(p)]for j in range(sample_size)])
h_vector = np.linalg.solve(z_left.T.dot(z_left), z_left.T.dot(my_d))
A_vector = [abs(h_vector[k]) for k in range(p)]
phi_vector = [math.atan(h_vector[i].imag / h_vector[i].real) for i in range(p)]


[1.1387, 1.09506, 1.04571, 0.991055]

[[1.09506 1.1387 ]
 [1.04571 1.09506]]

[-1.04571  -0.991055]


[0.32507006205899314, -0.3250700620589878]